# K-Nearest Neigbor

k-Nearest Neighbor makes predictions for each new point based on the points that are closest to the new point.
Contrary to other techniques that look at the data as a whole in order to learn patterns in the data, nearest neighbor only compares points closest to a new point. Therefore the technique won't be able to tell something about the drivers of a prediction. 

In [ ]:
from IPython.display import Image
Image(filename='/Users/annalie/Dev/data-science-from-scratch/pictures/knn-formula.png')

In [20]:
from __future__ import division
from collections import Counter
from linear_algebra import distance
from statistics import mean
import math, random
%matplotlib inline
import matplotlib.pyplot as plt

In [21]:
# for example, classify a new data point, based on the labeled points closest to the new data point

# create a function that counts the outcome, e.g. votes
def raw_majority_vote(labels):
    votes = Counter(labels)
    winner, _ = votes.most_common(1)[0]
    return winner

# reduce k to 1
def majority_vote(labels):
    """assumes that labels are ordered from nearest to farthest"""
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winners = len([count
                      for count in vote_counts.values()
                      if count == winner_count])
    if num_winners == 1:
        return                            # unique winner
    else:
        return majority_vote(labels[:-1]) # try again without the farthest data point

In [22]:
# knn classifier:
def knn_classify(k, labeled_points, new_point):
    """ each labeled point should be a pair (point label)"""
    
    # order the labeled pints from nearest to farthest
    by_distance = sorted(labeled_points, 
                        key = lambda (point, _): distance(point, new_point))
    
    # find the labels for the k closest
    k_nearest_labels = [label for _, label in by_distance[:k]]
    
    # reduce k to 1
    return majority_vote(k_nearest_labels)

# Practice example with library for KNN

In [45]:
import graphlab
from graphlab import SFrame
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

'''Check GraphLab Create version'''
from distutils.version import StrictVersion
assert (StrictVersion(graphlab.version) >= StrictVersion('1.8.5')), 'GraphLab Create must be version 1.8.5 or later.'

In [42]:
# load wikipedia dataset
wiki = graphlab.SFrame('/Users/annalie/Dev/clustering-retrieval/people_wiki.gl')
wiki

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


In [25]:
wiki[0]

{'URI': '<http://dbpedia.org/resource/Digby_Morrell>',
 'name': 'Digby Morrell',
 'text': 'digby morrell born 10 october 1979 is a former australian rules footballer who played with the kangaroos and carlton in the australian football league aflfrom western australia morrell played his early senior football for west perth his 44game senior career for the falcons spanned 19982000 and he was the clubs leading goalkicker in 2000 at the age of 21 morrell was recruited to the australian football league by the kangaroos football club with its third round selection in the 2001 afl rookie draft as a forward he twice kicked five goals during his time with the kangaroos the first was in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell was traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 games for the blues before being delisted at the end of 2005 he continu

In [64]:
# extract word count vectors
wiki['word_count'] = graphlab.text_analytics.count_words(wiki['text'])

In [65]:
wiki[0]

{'URI': '<http://dbpedia.org/resource/Digby_Morrell>',
 'name': 'Digby Morrell',
 'text': 'digby morrell born 10 october 1979 is a former australian rules footballer who played with the kangaroos and carlton in the australian football league aflfrom western australia morrell played his early senior football for west perth his 44game senior career for the falcons spanned 19982000 and he was the clubs leading goalkicker in 2000 at the age of 21 morrell was recruited to the australian football league by the kangaroos football club with its third round selection in the 2001 afl rookie draft as a forward he twice kicked five goals during his time with the kangaroos the first was in a losing cause against sydney in 2002 and the other the following season in a drawn game against brisbaneafter the 2003 season morrell was traded along with david teague to the carlton football club in exchange for corey mckernan he played 32 games for the blues before being delisted at the end of 2005 he continu

In [66]:
# create a model for knn that does not take into account the most too common words

# collect only the words that are not too common
wiki['tf_idf'] = graphlab.text_analytics.tf_idf(wiki['word_count'])

# create model
euclidian_tf_idf = graphlab.nearest_neighbors.create(wiki, label='name', features=['tf_idf'],
                                                 method='brute_force', distance='euclidean')

Starting brute force nearest neighbors model training.

In [67]:
# find the 10 nearest neighbors of Obama
euclidian_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=10)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 26.758ms     |

| Done         |         | 100         | 586.711ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
Barack Obama,Barack Obama,0.0,1
Barack Obama,Phil Schiliro,106.861013691,2
Barack Obama,Jeff Sessions,108.871674216,3
Barack Obama,Jesse Lee (politician),109.045697909,4
Barack Obama,Samantha Power,109.108106165,5
Barack Obama,Bob Menendez,109.781867105,6
Barack Obama,Eric Stern (politician),109.95778808,7
Barack Obama,James A. Guest,110.413888718,8
Barack Obama,Roland Grossenbacher,110.4706087,9
Barack Obama,Tulsi Gabbard,110.696997999,10


In [ ]:
# look for the most frequent used words between two persons
# and sort by weight

def top_words_tf_idf(name):
    row = wiki[wiki['name'] == name]
    word_count_table = row[['tf_idf']].stack('tf_idf', new_column_name=['word','weight'])
    return word_count_table.sort('weight', ascending=False)

In [ ]:
obama_tf_idf = top_words_tf_idf('Barack Obama')
schiliro_tf_idf = top_words_tf_idf('Phil Schiliro')
print "most frequent words used by Obama: ", obama_tf_idf
print "most frequent words used by Schiliro: ", schiliro_tf_idf

In [ ]:
# extract the rows from both tables that correspond to the common words
combined_words = obama_tf_idf.join(schiliro_tf_idf, on='word')

# rename the columns
combined_words = combined_words.rename({'weight':'Obama', 'weight.1':'Schiliro'})

# sort frequency of words by Obama
combined_words.sort('Obama', ascending=False)

combined_words

In [ ]:
# distance between Obama and Biden
obama = wiki[wiki['name'] == 'Barack Obama']
biden = wiki[wiki['name'] == 'Joe Biden']
graphlab.distances.euclidean(obama['tf_idf'][0],biden['tf_idf'][0])

In [ ]:
# compute the length of each Wikipedia document
# and examine the document lengths for the 100 nearest neighbors to Obama's page.

def compute_length(row):
    return len(row['text'].split(' '))

# define the length of each article
wiki['length'] = wiki.apply(compute_length) 
# create the model
nearest_neighbors_euclidean = euclidian_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
# join the columns of the euclidian distance and rank from the model with the wiki dataset for Obama
nearest_neighbors_euclidean = nearest_neighbors_euclidean.join(wiki[['name', 'length']], on={'reference_label':'name'})
# sort the dataset by closest euclidian distance
nearest_neighbors_euclidean.sort('rank')

In [ ]:
# compare the document length of Joe Biden to the lengths of other documents in the corpus

# make a histogram of the document lengths of Obama's 100 nearest neighbors
# and compare to a histogram of document lengths for all documents.
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])

plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size':16})
plt.tight_layout()

    Note: most of the nearest neighbors for Obama are articles with less than 300 words. 
    To remove this bias, we could choose for another distance, the cosine.
    Cosine distances compare word distributions of two articles of varying lengths.

In [ ]:
# retrain the model for cosine distance, instead of euclidian distance
cosine_tf_idf = graphlab.nearest_neighbors.create(wiki, label='name', features=['tf_idf'],
                                                  method='brute_force', distance='cosine')
nearest_neighbors_cosine = cosine_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
nearest_neighbors_cosine = nearest_neighbors_cosine.join(wiki[['name', 'length']], on={'reference_label':'name'})
nearest_neighbors_cosine.sort('rank')

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.hist(nearest_neighbors_cosine['length'], 50, color='b', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (cosine)', zorder=11, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])
plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size': 16})
plt.tight_layout()

    This graph shows more variability in the length of the documents that are the closest to Obama.

# Example with Twitter API and users from SSIReview

    When I read a magazin, I read the Stanford Social Innovation Review. 
    Since I like the magazine I'm interested in who else likes the magazine and tweets about it.
    What could I see about the similarities of these tweets using k-nearest neigbor?

In [35]:
from twython import Twython

In [36]:
consumer_key = 'k52czAojbrEf7hvOXgIlBpZRQ'
consumer_secret = 'ipAgKxOiC7wqG0KK0QInOcnf3XpuGbb8OV2KerWvDVZT1UXXUj'
twitter = Twython(consumer_key, consumer_secret)

In [54]:
import pandas as pd
from pandas import DataFrame
# search for tweets containing the phrase "SOCAP"
my_tweets = []
for status in twitter.search(q = '"ssireview"')["statuses"]:
    user = status["user"]["screen_name"].encode('utf-8')
    text = status["text"].encode('utf-8')
    my_tweets.append({'user': user, 'text': text})
tweets_df = DataFrame(my_tweets)

In [61]:
tweets_df

,text,user
0,RT @impacthubbalt: The New Economics of Innova...,InvestedImpact
1,RT @AECFNews: Philanthropies are beginning to ...,EPIPDC
2,Love this! -&gt; What would happen if we made ...,LA2050
3,The Social Labs Revolution: A New Approach to ...,JorgeGdelArco
4,"RT @accion: #Socent improves economies, genera...",GSE_Institute
5,Logic models and SMART goals won't end #reduce...,IFImpact
6,The Future of Banking for the Poor: Lessons fr...,MazarsCSV
7,“How can making a city more walkable improve e...,EqMeasure
8,@SSIReview I'm heavily involved in a project t...,ArtandContent
9,.@mfeigelson1 talks about designing our cities...,RebeccaWinthrop


In [56]:
sf = SFrame(data = tweets_df)
sf[0]

{'text': 'RT @impacthubbalt: The New Economics of Innovation Ecosystems (SSIR) https://t.co/IMTcAI2QOJ via @SSIReview',
 'user': 'InvestedImpact'}

In [58]:
# extract word count vectors
sf['word_count'] = graphlab.text_analytics.count_words(sf['text'])

In [59]:
# create a model for knn that does not take into account the most too common words

# collect only the words that are not too common
sf['tf_idf'] = graphlab.text_analytics.tf_idf(sf['word_count'])

In [60]:
# create model
sf_euclidian_tf_idf = graphlab.nearest_neighbors.create(sf, label='user', features=['tf_idf'],
                                                        method='brute_force', distance='euclidean')

Starting brute force nearest neighbors model training.

In [62]:
# find the 10 nearest neighbors of InvestedImpact
sf_euclidian_tf_idf.query(sf[sf['user'] == 'InvestedImpact'], label='user', k=10)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 6.66667     | 609us        |

| Done         |         | 100         | 760us        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
InvestedImpact,InvestedImpact,0.0,1
InvestedImpact,eric_hontz,0.0,2
InvestedImpact,impacthubbalt,2.29494730724,3
InvestedImpact,Bobincredible,7.5772965071,4
InvestedImpact,accion,9.06832332336,5
InvestedImpact,MazarsCSV,9.38155820239,6
InvestedImpact,GSE_Institute,9.40005717153,7
InvestedImpact,NPhealthcareAB,9.48805921317,8
InvestedImpact,JorgeGdelArco,10.091321004,9
InvestedImpact,EqMeasure,10.788974563,10


    Based on the output it seems like InvestedImpact and eric_hontz are tweeting exactly the same things.
    However, when looking at both profiles, I can't find the relation between these two accounts clearly.

In [69]:
# look for the most frequent used words between two persons
# and sort by weight

def top_words_tf_idf(user):
    row = sf[sf['user'] == user]
    word_count_table = row[['tf_idf']].stack('tf_idf', new_column_name=['word','weight'])
    return word_count_table.sort('weight', ascending=False)

InvestedImpact_tf_idf = top_words_tf_idf('InvestedImpact')
impacthubbalt_tf_idf = top_words_tf_idf('impacthubbalt')
print "most frequent words used by InvestedImpact: ", InvestedImpact_tf_idf
print "most frequent words used by impacthubbalt: ", impacthubbalt_tf_idf

# extract the rows from both tables that correspond to the common words
combined_words = InvestedImpact_tf_idf.join(impacthubbalt_tf_idf, on='word')

# rename the columns
combined_words = combined_words.rename({'weight':'InvestedImpact', 'weight.1':'impacthubbalt'})

# sort frequency of words by InvestedImpact
combined_words.sort('InvestedImpact', ascending=False)

combined_words

most frequent words used by InvestedImpact:  +-------------------------+----------------+
|           word          |     weight     |
+-------------------------+----------------+
|     @impacthubbalt:     | 2.01490302054  |
|        innovation       | 1.60943791243  |
|        ecosystems       | 1.60943791243  |
| https://t.co/imtcai2qoj | 1.60943791243  |
|        economics        | 1.60943791243  |
|            of           | 1.32175583998  |
|           new           | 1.32175583998  |
|          (ssir)         | 1.09861228867  |
|            rt           | 1.09861228867  |
|           the           | 0.916290731874 |
+-------------------------+----------------+
[12 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
most frequent words used by impacthubbalt:  +-------------------------+----------------+
|           word          |     weight     |
+-------------------------+--------------

word,InvestedImpact,impacthubbalt
innovation,1.60943791243,1.60943791243
ecosystems,1.60943791243,1.60943791243
https://t.co/imtcai2qoj,1.60943791243,1.60943791243
economics,1.60943791243,1.60943791243
of,1.32175583998,1.32175583998
new,1.32175583998,1.32175583998
(ssir),1.09861228867,1.09861228867
the,0.916290731874,0.916290731874
via,0.762140052047,0.762140052047
@ssireview,0.310154928304,0.310154928304


In [70]:
# compute the length of each Wikipedia document
# and examine the document lengths for the 100 nearest neighbors to Obama's page.

def compute_length(row):
    return len(row['text'].split(' '))

# define the length of each article
sf['length'] = sf.apply(compute_length) 
# create the model
sf_nearest_neighbors_euclidean = sf_euclidian_tf_idf.query(sf[sf['user'] == 'InvestedImpact'], label='user', k=100)
# join the columns of the euclidian distance and rank from the model with the wiki dataset for Obama
sf_nearest_neighbors_euclidean = sf_nearest_neighbors_euclidean.join(sf[['user', 'length']], on={'reference_label':'user'})
# sort the dataset by closest euclidian distance
sf_nearest_neighbors_euclidean.sort('rank')

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 6.66667     | 326us        |

| Done         |         | 100         | 464us        |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank,length
InvestedImpact,InvestedImpact,0.0,1,12
InvestedImpact,eric_hontz,0.0,2,12
InvestedImpact,impacthubbalt,2.29494730724,3,10
InvestedImpact,Bobincredible,7.5772965071,4,9
InvestedImpact,accion,9.06832332336,5,15
InvestedImpact,MazarsCSV,9.38155820239,6,15
InvestedImpact,GSE_Institute,9.40005717153,7,17
InvestedImpact,NPhealthcareAB,9.48805921317,8,13
InvestedImpact,JorgeGdelArco,10.091321004,9,17
InvestedImpact,EqMeasure,10.788974563,10,15


In [4]:
# access the Streaming API, which allows to connect to many more tweets

from twython import TwythonStreamer

tweets = []

class MyStreamer(TwythonStreamer):
    """our own subclass of TwythonStreamer that specifies
    how to interact with the stream"""
    
    def on_success(self, data):
        """the tweets that will be send by twitter will be collected in a dictionary"""
        
        # only collect English-language tweets
        if data['lang'] == 'en':
            tweets.append(data)
            print "received tweet #: ", len(tweets)
        
        # stop when we've collected enough
        if len(tweets) >= 50:
            self.disconnect()
            
    def on_error(self, status_code, data):
        print status_code, data
        self.disconnect()

In [ ]:
access_token = '236827879-5eXYWjODo5PdMsRd3yRSQUJTs0PPKcJY3t7GHy0r'
access_token_secret = 'PHgFDuempE9GdW8VUYfcTntakF1xCknILmKAIvcTstoaf'

stream = MyStreamer(consumer_key, consumer_secret, 
                    access_token, access_token_secret)

stream.statuses.filter(track = 'data')

In [ ]:
top_hashtags = Counter(hashtag['text'].lower()
                     for tweet in tweets
                     for hashtag in tweet["entities"]["hashtags"])

print top_hashtags.most_common(5)